In [9]:
import yfinance as yf
import pandas as pd
import sys
import time
import requests
import io
import urllib3

# Désactivation des alertes SSL pour assurer le chargement de l'univers
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_universe_list(choice):
    """Récupère les constituants des indices mondiaux via Wikipedia"""
    universes = {
        "1": ("S&P 500", "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies", 0, 'Symbol', 'GICS Sector'),
        "2": ("STOXX Europe 600", "https://en.wikipedia.org/wiki/STOXX_Europe_600", 2, 'Ticker', 'Sector'),
        "3": ("MSCI World (Proxy)", "https://en.wikipedia.org/wiki/MSCI_World", 2, 'Ticker', 'Sector')
    }
    
    if choice not in universes: return None, None, None
    name, url, table_idx, t_col, s_col = universes[choice]
    
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, verify=False)
        df = pd.read_html(io.StringIO(response.text))[table_idx]
        df[t_col] = df[t_col].astype(str).str.replace('.', '-', regex=False)
        return df[[t_col, s_col]], t_col, s_col
    except Exception as e:
        print(f"❌ Erreur Index: {e}")
        return None, None, None

def run_eqs():
    print("="*100)
    print("   TERMINAL BLOOMBERG | EQUITY SCREENING (EQS) - PROFESSIONAL DATAFRAME")
    print("="*100)
    
    idx_choice = input("\nSélectionnez l'univers : 1 (S&P 500), 2 (STOXX 600), 3 (MSCI WORLD) ")
    
    df_univ, t_col, s_col = get_universe_list(idx_choice)
    if df_univ is None: return

    # Affichage clair de la liste des secteurs pour éviter de deviner
    sectors = sorted(df_univ[s_col].unique().tolist())
    print(f"\n--- SECTEURS DISPONIBLES ({len(sectors)}) ---")
    for i, s in enumerate(sectors): print(f"{i:2d}. {s}")
    
    try:
        sec_idx = int(input("\n👉 Entrez le numéro du secteur à analyser : "))
        target_sector = sectors[sec_idx]
    except: return

    tickers = df_univ[df_univ[s_col] == target_sector][t_col].tolist()
    print(f"\n🚀 Scan de {len(tickers)} valeurs... (Données Financières Complètes)\n")

    raw_data = []
    for i, ticker in enumerate(tickers):
        try:
            sys.stdout.write(f"\rExtraction {i+1}/{len(tickers)} : {ticker:<10}")
            sys.stdout.flush()
            
            tk = yf.Ticker(ticker)
            info = tk.info
            cf = tk.cashflow # Récupération des rapports annuels pour le FCF historique

            # Gestion du PEG et Earnings Growth (Justifie les 'None' en cas d'absence)
            peg = info.get('pegRatio') or info.get('trailingPegRatio')
            earn_growth = info.get('earningsGrowth', 0) * 100 if info.get('earningsGrowth') else None
            
            # Analyse FCF 1Y/3Y Moyenne
            fcf_3y, fcf_1y = None, None
            if not cf.empty and 'Free Cash Flow' in cf.index:
                fcf_series = cf.loc['Free Cash Flow']
                fcf_1y = fcf_series.iloc[0] / 1e9 if len(fcf_series) > 0 else None
                fcf_3y = fcf_series.iloc[:3].mean() / 1e9 if len(fcf_series) >= 3 else None

            raw_data.append({
                # --- IDENTIFICATION & MARCHÉ ---
                'Symbol': ticker,
                'Name': info.get('shortName', 'N/A')[:15],
                'Mkt Cap (B)': info.get('marketCap', 0) / 1e9 if info.get('marketCap') else None,
                
                # --- VALUATION (Coherent Grouping) ---
                'P/E (TTM)': info.get('trailingPE'),
                'PEG Ratio': peg,
                'Earn Gr %': earn_growth,
                'EV/EBITDA': info.get('enterpriseToEbitda'),
                'P/Book': info.get('priceToBook'),
                
                # --- CASH FLOW & RENDEMENT ---
                'FCF 1Y (B)': fcf_1y,
                'FCF 3Y (B)': fcf_3y,
                'FCF Yield %': (info.get('freeCashflow', 0) / info.get('marketCap', 1)) * 100 if info.get('freeCashflow') and info.get('marketCap') else None,
                'Div Yield %': info.get('dividendYield', 0) * 100 if info.get('dividendYield') else None,
                
                # --- RENTABILITÉ & SOLVABILITÉ ---
                'ROE %': info.get('returnOnEquity', 0) * 100 if info.get('returnOnEquity') else None,
                'Oper Margin %': info.get('operatingMargins', 0) * 100 if info.get('operatingMargins') else None,
                'D/E Ratio': info.get('debtToEquity'),
                'Current Ratio': info.get('currentRatio'),
                
                # --- CROISSANCE & PAYOUT ---
                'Payout %': info.get('payoutRatio', 0) * 100 if info.get('payoutRatio') else None,
                'Rev Gr %': info.get('revenueGrowth', 0) * 100 if info.get('revenueGrowth') else None,
            })
            if len(tickers) > 20: time.sleep(0.1) # Sécurité API
        except: continue

    # Génération du DataFrame final
    df_res = pd.DataFrame(raw_data)
    
    if not df_res.empty:
        # Tri par Market Cap décroissant
        df_res = df_res.sort_values('Mkt Cap (B)', ascending=False).reset_index(drop=True)
        
        # Configuration de l'affichage pro
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_rows', None)
        
        print("\n\n" + "="*180)
        print(f"DATAFRAME EQS : {target_sector.upper()} | UNIVERS : {idx_choice}")
        print("="*180)
        # Arrondi à 2 décimales pour la propreté visuelle
        print(df_res.round(2).to_string())
        print("="*180)
    else:
        print("\n⚠️ Extraction vide ou erreur de récupération.")

if __name__ == "__main__":
    run_eqs()

   TERMINAL BLOOMBERG | EQUITY SCREENING (EQS) - PROFESSIONAL DATAFRAME

--- SECTEURS DISPONIBLES (11) ---
 0. Communication Services
 1. Consumer Discretionary
 2. Consumer Staples
 3. Energy
 4. Financials
 5. Health Care
 6. Industrials
 7. Information Technology
 8. Materials
 9. Real Estate
10. Utilities

🚀 Scan de 36 valeurs... (Données Financières Complètes)

Extraction 36/36 : WMT       

DATAFRAME EQS : CONSUMER STAPLES | UNIVERS : 1
   Symbol             Name  Mkt Cap (B)  P/E (TTM)  PEG Ratio  Earn Gr %  EV/EBITDA  P/Book  FCF 1Y (B)  FCF 3Y (B)  FCF Yield %  Div Yield %   ROE %  Oper Margin %  D/E Ratio  Current Ratio  Payout %  Rev Gr %
0     WMT     Walmart Inc.       888.26      38.95       4.05       35.2      22.29    9.24       12.66       13.25         1.17         84.0   23.66           3.73      66.75           0.80     31.91       5.8
1    COST  Costco Wholesal       382.84      46.09       4.80       11.4      28.48   12.63        7.84        7.07         1.87   